In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
from glob import glob
import os.path
from matplotlib.pyplot import subplots
import numpy as np


%matplotlib inline

# Retrieve activity data from ChEMBL

In [2]:
import urllib
import json
import re

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def get_ligands(accession):

    data = []

    target_data = json.loads(urllib.urlopen("https://www.ebi.ac.uk/chemblws/targets/uniprot/%s.json" %
                                             accession).read())
    bioactivity_data = json.loads(urllib.urlopen("https://www.ebi.ac.uk/chemblws/targets/%s/bioactivities.json" %
                                                  target_data['target']['chemblId']).read())

    for bioactivity in bioactivity_data['bioactivities']:
        
        if not is_number(bioactivity["value"]):
            continue

        row = {}
        
        for key in ["bioactivity_type", "operator", "value", "units", "target_chemblid",
                    "ingredient_cmpd_chemblid"]:
            row[key] = bioactivity[key]
        data.append(row)

    return data

In [4]:
names = []

if os.path.exists("../data/raw_data.csv"):
    all_data = pd.read_csv("../data/raw_data.csv", index_col=0)
    for i in glob("../data/targets/*"):
        name = i.split("/")[-1][:-4]
        print(name)
        names.append(name)

else:
    all_data = pd.DataFrame(columns=["name", "uniprot_name", "target_chemblid", "bioactivity_type", "operator",
                                     "value", "units", "ingredient_cmpd_chemblid"])

    for i in glob("../data/targets/*"):
        name = i.split("/")[-1][:-4]
        print(name)
        names.append(name)
        f = open(i)
        for line in f:
            tmp = line.split("\t")
            accession = tmp[0]
            uniprot_name = tmp[1]
            try:
                tmp_data = get_ligands(accession)
                if tmp_data:
                    tmp_pd = pd.DataFrame(data=tmp_data, columns=["name", "uniprot_name", "target_chemblid",
                                                                  "units", "bioactivity_type", "operator",
                                                                  "value", "ingredient_cmpd_chemblid"])
                    tmp_pd["name"] = name
                    tmp_pd["uniprot_name"] = uniprot_name

                    all_data = pd.concat([all_data, tmp_pd], ignore_index=True)
            except:
                continue

        f.close()
    all_data.to_csv("../data/raw_data.csv")

CHRNA10
NPY2R
ADORA2A
HTR1A
MC3R
SLC6A3
MC4R
ESR1
PTGS1
SLC29A1
SCN5A
CRHR1
HRH1
HTR3A
ADRA2B
NR1I2
NR3C1
MLN
DRD1
EDNRA
SLC6A2
AVPR2
ADRA2C
PGR
DRD3
GRIN3A
GABRA1
PDE4D
PTGS2
CHRM3
HTR2B
NPY1R
CHRM2
DRD4
SLC6A4
HTR2A
ADRA2A
ADRB1
CACNA1C
AGTR1
CNR1
GIPR
EDNRB
GHSR
OPRK1
HRH2
CACNA1B
PDE3A
CCKBR
CCKAR
GPR109A
ADORA3
ADRB3
CHRM1
ADORA1
TACR1
ESR2
MAOA
KCNH2
BDKRB1
DRD2
OPRD1
ADRA1A
AR
ADRB2
BDKRB2
HRH3
TBXA2R
HTR2C
AVPR1A
NTSR1
OPRM1
CRHR2


In [ ]:
all_data["bioactivity_type"].unique()

In [ ]:
all_data["units"].unique()

In [ ]:
len(all_data["uniprot_name"].unique())

In [ ]:
all_data["value"] = all_data["value"].astype(float)

In [ ]:
idx_op = ((all_data["operator"] == "<") | (all_data["operator"] == "<=") |
          (all_data["operator"] == "=") | (all_data["operator"] == "~"))

idx_type = ((all_data["bioactivity_type"] == "IC50") | (all_data["bioactivity_type"] == "EC50") |
            (all_data["bioactivity_type"] == "Ki") | (all_data["bioactivity_type"] == "Kd"))

idx_val = (all_data["value"] <= 1000)
idx_units = (all_data["units"] == "nM")
idx_above0 = (all_data["value"] >= 0)

In [ ]:
cleaned_data = all_data[idx_op & idx_type & idx_val & idx_units & idx_above0]
cleaned_data.to_csv("../data/cleaned_data.csv")

# Get SMILES

In [ ]:
to_retrieve = list(cleaned_data["ingredient_cmpd_chemblid"].unique())

In [ ]:
len(to_retrieve)

In [ ]:
def get_smiles(chembl_id):
    cmpd_data = json.loads(urllib2.urlopen("https://www.ebi.ac.uk/chemblws/compounds/%s.json" %
                                           chembl_id).read())
    if "smiles" not in cmpd_data["compound"]:
        return None
    else:
        return cmpd_data['compound']["smiles"]

In [ ]:
if os.path.exists("../data/chembl_smiles.csv"):
    cmpd_smiles = pd.read_csv("../data/chembl_smiles.csv", index_col = 0)

else:
    smiles = []
    for i in xrange(len(to_retrieve)):
        if i%1000 == 0:
            print i
        smiles.append(get_smiles(to_retrieve[i]))
    cmpd_smiles = pd.DataFrame({"SMILES": smiles, "ingredient_cmpd_chemblid": to_retrieve})
    cmpd_smiles.to_csv("../data/chembl_smiles.csv")

In [ ]:
without_smiles = (cmpd_smiles["SMILES"].isnull())
sum(without_smiles)

In [ ]:
cmpd_with_smiles = cmpd_smiles.dropna()
with_smiles = pd.merge(cleaned_data, cmpd_with_smiles)

# Filter out long peptides

In [ ]:
from pybel import Smarts, readstring, readfile
pept_bond = Smarts("[$([NX3H2,NX4H3+]),$([NX3H](C)(C))][CX4H]([*])[CX3](=[OX1])[OX2H,OX1-,N]")

In [ ]:
def pept_len(x):
    # number of aminoacids in a molecule
    return len(pept_bond.findall(readstring("smi", x)))

In [ ]:
with_smiles["pept"] = with_smiles.apply(lambda x: pept_len(str(x["SMILES"])), axis=1)

In [ ]:
idx_nonpept = (with_smiles["pept"] < 20)
with_smiles = with_smiles[idx_nonpept]

In [ ]:
cleaned_data.shape, with_smiles.shape

In [ ]:
targets_groups = with_smiles.groupby(["name"])
cmpd_uniq_counts = targets_groups.aggregate({"ingredient_cmpd_chemblid": lambda x: len(x.unique())})
cmpd_uniq_counts.columns = ["counts"]
cmpd_uniq_counts.head()

In [ ]:
cmpd_uniq_counts["counts"].min(), cmpd_uniq_counts["counts"].argmin()

In [ ]:
f, ax = subplots(figsize=(3.35,3), dpi=300)
sns.distplot(cmpd_uniq_counts["counts"], bins=range(0, 7000, 500), kde=False, ax=ax)
ax.set_xlabel("Number of active ligands")
ax.set_ylabel("Number of targets")

f.tight_layout()
f.savefig("../figures/size_distribution.pdf")

In [ ]:
for f in [np.min, np.mean, np.max, np.std]:
    print f.__name__, f(cmpd_uniq_counts["counts"])

In [ ]:
q = [5, 25,50,75, 95]
print "5th, 25th, 50th, 75th, and 95th percentiles:", np.percentile(cmpd_uniq_counts["counts"], q)

In [ ]:
with_smiles.to_csv("../data/cleaned_data_smiles.csv")

In [ ]:
with_smiles.head()

In [ ]:
for i in names:
    print i
    tmp = with_smiles[with_smiles["name"] == i][["SMILES", "ingredient_cmpd_chemblid"]].drop_duplicates()
    tmp.to_csv("../data/smi_files/"+i+".ism", sep="\t", header=False, index=False)